In [1]:
!python --version
!pip --version

Python 3.7.16
pip 22.3.1 from C:\Users\inoo3\Anaconda3\envs\kobert0\lib\site-packages\pip (python 3.7)



In [2]:
!pip list

Package              Version
-------------------- ---------
anyio                3.6.2
argon2-cffi          21.3.0
argon2-cffi-bindings 21.2.0
attrs                23.1.0
backcall             0.2.0
beautifulsoup4       4.12.2
bleach               6.0.0
certifi              2022.12.7
cffi                 1.15.1
colorama             0.4.6
debugpy              1.6.7
decorator            5.1.1
defusedxml           0.7.1
entrypoints          0.4
fastjsonschema       2.16.3
idna                 3.4
importlib-metadata   6.6.0
importlib-resources  5.12.0
ipykernel            6.16.2
ipython              7.34.0
ipython-genutils     0.2.0
ipywidgets           8.0.6
jedi                 0.18.2
Jinja2               3.1.2
jsonschema           4.17.3
jupyter              1.0.0
jupyter_client       7.4.9
jupyter-console      6.6.3
jupyter_core         4.12.0
jupyter-server       1.24.0
jupyterlab-pygments  0.2.2
jupyterlab-widgets   3.0.7
MarkupSafe           2.1.2
matplotlib-inline    0.1.6
mistune  

In [ ]:
%condalist

## **데이터 불러오기**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 밑에서 상대경로 안되면 sentiment-analysis 까지 이동
# %cd /content/drive/MyDrive/Github/floread/sentiment-analysis  #코랩
%cd /sentiment-analysis

[WinError 2] 지정된 파일을 찾을 수 없습니다: '/sentiment-analysis'
c:\gitDefi\floread\sentiment-analysis


In [4]:
import pandas as pd

train_set = pd.read_csv('data/감성대화말뭉치(병합).csv', index_col=0) #첫 번째 열이 인덱스 열일 때 Unnamed: 0 빼기
train_set.sample(n=5)

,sentence,emotion
9060,요즘 대인 관계 때문에 마비된 기분이야.,슬픔
40424,황혼이혼을 하더니 친구가 오히려 더 좋아진 것 같아.,분노
3055,언니가 엄마 치료비 내기 싫다고 연락을 끊어 버렸어.,상처
18075,몸이 아팠지만 같이 노인정에 가자는 김 할머니의 부탁을 거절하지 못했어.,불안
9696,진행하던 일이 드디어 성과를 냈어.,기쁨


In [ ]:
# train_set = train_set[['감정_대분류','사람문장1']]
# train_set.columns = ['emotion', 'sentence']  # 칼럼이름 변경

# train_set.head()

**감정을 정수 라벨로 변경**

In [5]:
# 감정을 정수 라벨로 변경
emotions = {'기쁨': 0, '불안': 1, '당황': 2, '슬픔': 3, '분노': 4, '상처': 5}

train_set['emotion'] = train_set.emotion.map(emotions)

train_set.sample(n=5)
# train_set.loc[(train_set['emotion'] == "기쁨"), 'emotion'] = 0  #기쁨 => 0
# train_set.loc[(train_set['emotion'] == "불안"), 'emotion'] = 1  #불안 => 1
# train_set.loc[(train_set['emotion'] == "당황"), 'emotion'] = 2  #당황 => 2
# train_set.loc[(train_set['emotion'] == "슬픔"), 'emotion'] = 3  #슬픔 => 3
# train_set.loc[(train_set['emotion'] == "분노"), 'emotion'] = 4  #분노 => 4
# train_set.loc[(train_set['emotion'] == "상처"), 'emotion'] = 5  #상처 => 5

,sentence,emotion
5540,취업 준비하는 거 너무 성가시다.,4
758,오늘 집에 오니 내가 그렇게 바라던 강아지가 있더라고! 너무 행복했어!,0
11539,내가 드디어 임신에 성공했어.,0
645,마을 대표로 노래자랑을 나가라는 추천을 받았어.,0
44923,갱년기 때문인지 요즘 부쩍 우울하고 몸이 예전 같지 않아. 자꾸 내가 초라해 보여.,5


In [8]:
!conda -m pip install --upgrade pip

SyntaxError: invalid syntax (<ipython-input-8-0a8cf7b99133>, line 1)

__Korean BERT pre-trained cased (KoBERT) for Huggingface Transformers__
<br>
koBERT Hugging Face 토크나이저 및  Requirements 설치
https://github.com/SKTBrain/KoBERT/blob/master/kobert_hf/requirements.txt


In [6]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install sentencepiece transformers
!pip install torch

!pip install mxnet
# !pip install -U --pre "mxnet-cu118>=2.0.0a"   # cu118이랑 맞는 버전 없음
# !pip install -U --pre "mxnet-cu118>=2.0.0a"
# !pip install mxnet-cu111==1.8.0
# !pip install mxnet-cu111mkl==1.8.0
# !pip install -U --pre "mxnet>=2.0.0a"   #cpu only
# !pip install mxnet

!pip install gluonnlp
!pip install tqdm   

ERROR: Invalid requirement: "'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer"
Hint: = is not a valid operator. Did you mean == ?
You should consider upgrading via the 'C:\Users\inoo3\Anaconda3\python.exe -m pip install --upgrade pip' command.
'subdirectory'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


     ------------------------------------- 977.9/977.9 KB 20.6 MB/s eta 0:00:00
     ---------------------------------------- 7.0/7.0 MB 34.2 MB/s eta 0:00:00
     ------------------------------------- 224.5/224.5 KB 13.4 MB/s eta 0:00:00
     ------------------------------------- 268.0/268.0 KB 16.1 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 44.3 MB/s eta 0:00:00
     ---------------------------------------- 77.1/77.1 KB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.36.1
    Uninstalling tqdm-4.36.1:
      Successfully uninstalled tqdm-4.36.1


You should consider upgrading via the 'C:\Users\inoo3\Anaconda3\python.exe -m pip install --upgrade pip' command.


     -------------------------------------- 162.6/162.6 MB 8.6 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\inoo3\Anaconda3\python.exe -m pip install --upgrade pip' command.


     --------------------------------------- 33.1/33.1 MB 29.8 MB/s eta 0:00:00
     --------------------------------------- 11.9/11.9 MB 36.4 MB/s eta 0:00:00
     ---------------------------------------- 88.7/88.7 KB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 56.5/56.5 KB 3.1 MB/s eta 0:00:00
     -------------------------------------- 132.3/132.3 KB 7.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.2
    Uninstalling urllib3-1.24.2:
      Successfully uninstalled urllib3-1.24.2
  Attempting uninstall: idna
    Found existing installation: idna 2.8
    Uninstalling idna-2.8:
      Successfully uninstalled idna-2.8
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled nu

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\users\\inoo3\\anaconda3\\lib\\site-packages\\~-mpy\\.libs\\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Users\inoo3\Anaconda3\python.exe -m pip install --upgrade pip' command.


     ------------------------------------- 344.5/344.5 KB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-win_amd64.whl size=465958 sha256=eb4e5c60caac0f27f9977d6364c40f8fba801707b7eab7dfee84ee94fdee6fac
  Stored in directory: c:\users\inoo3\appdata\local\pip\cache\wheels\be\b4\06\7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


You should consider upgrading via the 'C:\Users\inoo3\Anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\inoo3\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install torch==1.11.0+cu102 torchvision==0.12.0+cu102 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu102

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu102
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 49.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0
    Uninstalling torch-1.11.0:
      Successfully uninstalled torch-1.11.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0
    Uninstalling torchvision-0.12.0:
      Successfully uninstalled torchvision-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.15.1 requires torch==2.0.0, but you have torch 1.11.0+cu102 which is incompatible.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 1.11.0+cu102 which is incompatible.


In [4]:
!pip install -U --pre "mxnet-cu102>=2.0.0a"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.8/365.8 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1


In [6]:
import mxnet
print(mxnet.__version__)

OSError: ignored

In [7]:
# 코랩에서 사용하는 cuda버전
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
!pip install gluonnlp
import gluonnlp as nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=681097 sha256=af6927d98a5982bed2272382699e108d980ef3cb5b4e88d9d6ec4b1737e75f20
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


OSError: ignored

In [5]:
# kobert requirements
import torch
#import gluonnlp as nlp
# import transformers
# import sentencepiece

print("torch version:", torch.__version__)
# print("transformers version:", transformers.__version__)
# print("sentencepiece version:", sentencepiece.__version__)
#print("gluonnlp version:",nlp.__version__)

torch version: 1.11.0+cu102


In [ ]:
# 파이썬 버전
import platform
print(platform.python_version())

3.10.11


In [ ]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
import gluonnlp as nlp

#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = torch.device("cuda:0")

ImportError: ignored

In [ ]:
platform.python_version()

'3.10.11'

**토크나이저, pretrained 모델, vocabulary 로드**

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

# Setting parameters(KoBERT finetuning 베에스 라인을 그대로 따랐음)
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

NameError: name 'KoBERTTokenizer' is not defined

### **KOBert 클래스 정의**

In [ ]:
# 모델에 사용되는 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

In [ ]:
# 감성 분류 모델 정의
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, #클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

**정확도 계산**

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 입력에 대한 예측 반환
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

**데이터 셋 피팅**

In [ ]:
# 모델 학습에 사용할 데이터셋을 [data, label] 배열로 피팅
train_set_data = [[i, str(j)] for i, j in zip(train_set['sentence'], train_set['emotion'])]

# sklearn 의 train_test_split 모듈-> 4:1로 학습&검증 데이터를 분류 
train_set_data, test_set_data = train_test_split(train_set_data, test_size = 0.2, random_state=4)

# 데이터셋을 Bert모델에 입력할 수 있게 변환
train_set_data = BERTDataset(train_set_data, 0, 1, tok, vocab, max_len, True, False)
test_set_data = BERTDataset(test_set_data, 0, 1, tok, vocab, max_len, True, False)

# 배치데이터셋 생성
train_dataloader = torch.utils.data.DataLoader(train_set_data, batch_size=batch_size, num_workers=2)    # num_workers: 데이터 로딩할때 쓰는 프로세스 수
test_dataloader = torch.utils.data.DataLoader(test_set_data, batch_size=batch_size, num_workers=2)

학습 코드

In [ ]:
# 학습
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):   # 아까 만든 테스트 배치 데이터 - 정확도 측정

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

**모델 정보**

In [ ]:
# 모델 정보
print(type(model), model, sep="\n\n")

In [ ]:
# 상대경로 안됨?
# torch.save(model, './model/sa-tutorial1.pt')
# torch.save(model.state_dict(), './model/sa-tutorial1_StateDict.pt')

torch.save(model, f'/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1.pt')
torch.save(model.state_dict(), f'/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1_StateDict.pt')

In [ ]:
# 학습된 모델 저장
import os

model_path1 = '/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1.pt'
model_path2 = '/content/drive/MyDrive/Github/sentiment-analysis/model/sa-tutorial1_StateDict.pt'

size1 = os.path.getsize(model_path1) / (1024*1024) # mb 단위
size2 = os.path.getsize(model_path2) / (1024*1024) 
print(f"Model size: {size1:.2f} MB")
print(f"Model_StateDict size: {size2:.2f} MB")